### Interactive Assistant Client App

Trace (track down) detected person actions

In [1]:
import mysql.connector
import pandas as pd
import re

import requests
import json
import time
from os.path import join

In [2]:
# variables de conexión to DataBase
host_= "192.168.M.MMM" 
user_="db_user"
passwd_="db_user_password"
database_="database_name"

In [3]:
conn = mysql.connector.connect(host=host_, user=user_, passwd=passwd_, database=database_)
cursor = conn.cursor()
df = pd.read_sql("select * from sentiment2 where ddate = current_date() and time >= subtime(current_time(),000003) order by time DESC", conn)
df

,id,name,gender,age,neutral,happiness,surprise,sadness,anger,disgust,fear,contempt,time,ddate
0,381820,ID 36,,31.0,0.528475,1.444481e-05,0.000382,0.470832,0.000033,0.000126,0.000130,6.327317e-06,10:39:25,2019-02-22
1,381814,ID 36,,31.0,0.578774,1.007132e-05,0.001253,0.419213,0.000280,0.000119,0.000342,8.332042e-06,10:39:24,2019-02-22
2,381815,ID 36,,31.0,0.561397,8.755001e-06,0.000976,0.436982,0.000174,0.000111,0.000343,7.611499e-06,10:39:24,2019-02-22
3,381816,ID 36,,31.0,0.546686,1.246787e-05,0.000748,0.452021,0.000096,0.000146,0.000285,5.700642e-06,10:39:24,2019-02-22
4,381817,ID 36,,31.0,0.533981,1.310325e-05,0.000488,0.465115,0.000066,0.000120,0.000212,4.251515e-06,10:39:24,2019-02-22
5,381818,ID 36,,31.0,0.543975,1.628508e-05,0.000412,0.455284,0.000045,0.000114,0.000150,4.334500e-06,10:39:24,2019-02-22
6,381819,ID 36,,31.0,0.536930,1.688497e-05,0.000419,0.462256,0.000054,0.000148,0.000171,6.406763e-06,10:39:24,2019-02-22
7,381813,ID 24,,31.0,0.565296,2.178598e-06,0.000126,0.434335,0.000108,0.000030,0.000094,1.056428e-05,10:39:23,2019-02-22
8,381800,ID 26,,31.0,0.443322,7.282360e-07,0.000443,0.555964,0.000018,0.000030,0.000222,3.039064e-07,10:39:22,2019-02-22
9,381801,ID 26,,31.0,0.492058,2.233372e-06,0.001017,0.506563,0.000017,0.000082,0.000260,1.400889e-06,10:39:22,2019-02-22


In [4]:
list_detected = df["name"].value_counts()

Lógica para detectar y verificar que el reto se está dando sobre la misma persona

In [5]:
# Url service that interacts with current person in front of the screen
URL = "http://127.0.0.1:5000/"
ENDPOINT = join(URL, "test")
headers = {'Content-Type': 'application/json; charset=UTF-8'}

In [6]:
# Sentiment detection
def detectSentiment(sentiment, name, reto):
    time.sleep(0)

    success = "No"
    timeout = time.time() + 3
    while time.time() < timeout:

        conn = mysql.connector.connect(host=host_, user=user_, passwd=passwd_, database=database_)
        cursor = conn.cursor()
        df = pd.read_sql("select * from sentiment2 where ddate = current_date() and time >= subtime(current_time(),000004) order by time DESC", conn)

        scores = list(df[sentiment][df["name"]==name])
        print("scores: ", scores)
        
        if len(scores) > 0:
            if (max(scores) >= 0.6):
                score = "positive"
                reto = reto
                action = "input.feedback"
                success = "Yes"
                break
            else:
                score = "negative"
                reto = reto
                action = "input.feedback"
                success = "No"
        else:
            score = "negative"
            reto = reto
            action = "input.terminate"
            success = "No"
            
    return success, score, reto, action

In [7]:
# Check current person
def checkCurrentPerson(currentName):
    stillthere = "No"
    timeout = time.time() + 1
    while time.time() < timeout:

        conn = mysql.connector.connect(host=host_, user=user_, passwd=passwd_, database=database_)
        cursor = conn.cursor()
        df = pd.read_sql("select * from sentiment2 where ddate = current_date() and time >= subtime(current_time(),000004) order by time DESC", conn)

        list_detected = df["name"].value_counts()
        
        if list_detected.count() > 0:
            names = list(list_detected.index)
            if currentName in names:
                stillthere = "Yes"
                break
        else: stillthere = "No"
            
    return stillthere

In [8]:
# Only interact with the ones that have real names (not ID_43)
def notRealName(inputString):
    return bool(re.search(r'\d', inputString))

In [ ]:
# Real time connection to DB
detected = "No"

action = 'input.welcome'
name = ""
score = ""
reto = ""
state = ""

session = "end"

repeat = 0
fail = 0
listReto = []

while True:
    if session == "end" and detected == "No":
        conn = mysql.connector.connect(host=host_, user=user_, passwd=passwd_, database=database_)
        cursor = conn.cursor()
        df = pd.read_sql("select * from sentiment2 where ddate = current_date() and time >= subtime(current_time(),000003) order by time DESC", conn)
        
        list_detected = df["name"].value_counts()
        
        if list_detected.count() > 0:
            curr_names = list(list_detected.index)
            name = ""
            for i in range(len(curr_names)):
                if notRealName(curr_names[i]):
                    pass
                else:
                    name = curr_names[i]
                    break
            
            if name != "":
                detected = "Yes"
                action = "input.welcome"
                repeat = 0
                fail = 0
                listReto = []
            else: detected = "No"
        else: detected = "No"
    else:
        data = json.dumps({'name':name, 'action':action, 'score':score, 'reto':reto, 'repeat':repeat, 'fail':fail, 
                           'listReto':listReto, 'session':session}).encode('utf-8')
        response = requests.post(ENDPOINT, headers = headers, data=data)
        result = response.json()
        print("Result:", result)
        
        session = result['fulfillmentText']['session']
        detected = result['fulfillmentText']['detected']
        state = result['fulfillmentText']['state']
        
        if state == "welcome":
            #time.sleep(1)
            stillthere = checkCurrentPerson(name)
            if stillthere == "Yes":
                action = "input.challenge"
            else:
                action = "input.terminate"
        
        if state == "challenge":
            reto = result['fulfillmentText']['reto']
            if reto == "happiness":
                sentiment = reto
                listReto.append(reto)
                succ, score, reto, action = detectSentiment(sentiment, name, reto)
                if succ == "Yes":
                    repeat += 1
                else: fail += 1
                        
            elif reto == "surprise":
                sentiment = reto
                listReto.append(reto)
                succ, score, reto, action = detectSentiment(sentiment, name, reto)
                if succ == "Yes":
                    repeat += 1
                else: fail += 1
            
            elif reto == "sadness":
                sentiment = reto
                listReto.append(reto)
                succ, score, reto, action = detectSentiment(sentiment, name, reto)
                if succ == "Yes":
                    repeat += 1
                else: fail += 1
            
            elif reto == "anger":
                sentiment = reto
                listReto.append(reto)
                succ, score, reto, action = detectSentiment(sentiment, name, reto)
                if succ == "Yes":
                    repeat += 1
                else: fail += 1
            
            elif reto == "disgust":
                sentiment = reto
                listReto.append(reto)
                succ, score, reto, action = detectSentiment(sentiment, name, reto)
                if succ == "Yes":
                    repeat += 1
                else: fail += 1
            
            elif reto == "fear":
                sentiment = reto
                listReto.append(reto)
                succ, score, reto, action = detectSentiment(sentiment, name, reto)
                if succ == "Yes":
                    repeat += 1
                else: fail += 1
            else: pass
        
        if state == "terminate":
            time.sleep(8)
            pass
